<a target="_blank" href="https://colab.research.google.com/github/halsawadi/llms-bootcamp/blob/master/PromptEngineering.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Prompt Engineering

# 1. Guidelines for Prompting
In this lesson, you'll practice two prompting principles and their related tactics in order to write effective prompts for large language models.

## Setup
#### Load the API key and relevant Python libaries.

In this course, we've provided some code that loads the OpenAI API key for you.

In [3]:
import cohere
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

cohere.api_key  = None

#### helper function
Throughout this course, we will use OpenAI's `gpt-3.5-turbo` model and the [chat completions endpoint](https://platform.openai.com/docs/guides/chat). 

This helper function will make it easier to use prompts and look at the generated outputs:

In [51]:
def get_completion(prompt, model="command-xlarge-20221108"):
    co = cohere.Client(cohere.api_key) # This is your trial API key
    response = co.generate(
        model='command',
        prompt=prompt,
        max_tokens=700,
        temperature=0)
    return response.generations[0].text

## Prompting Principles
- **Principle 1: Write clear and specific instructions**
- **Principle 2: Give the model time to “think”**

### Tactics

### Principle 1: Write clear and specific instructions 
#### Tactic 1: Use delimiters to clearly indicate distinct parts of the input
- Delimiters can be anything like: ```, """, < >, `<tag> </tag>`, `:`

In [15]:
text = f"""
You should express what you want a model to do by \ 
providing instructions that are as clear and \ 
specific as you can possibly make them. \ 
This will guide the model towards the desired output, \ 
and reduce the chances of receiving irrelevant \ 
or incorrect responses. Don't confuse writing a \ 
clear prompt with writing a short prompt. \ 
In many cases, longer prompts provide more clarity \ 
and context for the model, which can lead to \ 
more detailed and relevant outputs.
"""
prompt = f"""
Summarize the text delimited by triple backticks \ 
into a single sentence.
```{text}```
"""
response = get_completion(prompt)
print(response)


Instructions that are clear and specific guide the model towards the desired output, and reduce the chances of receiving irrelevant or incorrect responses.


#### Tactic 2: Ask for a structured output
- JSON, HTML

In [16]:
prompt = f"""
Generate a list of three made-up book titles along \ 
with their authors and genres. 
Provide them in JSON format with the following keys: 
book_id, title, author, genre.
"""
response = get_completion(prompt)
print(response)


{
"book_id": 1,
"title": "The Magic Carpet",
"author": "Mary Shelley",
"genre": "Fiction"
},
{
"book_id": 2,
"title": "The Time Machine",
"author": "H.G. Wells",
"genre": "Fiction"
},
{
"book_id": 3,
"title": "The War of the Worlds",
"author": "H.G. Wells",
"genre": "Fiction"
}


#### Tactic 3: Ask the model to check whether conditions are satisfied

In [17]:
text_1 = f"""
Making a cup of tea is easy! First, you need to get some \ 
water boiling. While that's happening, \ 
grab a cup and put a tea bag in it. Once the water is \ 
hot enough, just pour it over the tea bag. \ 
Let it sit for a bit so the tea can steep. After a \ 
few minutes, take out the tea bag. If you \ 
like, you can add some sugar or milk to taste. \ 
And that's it! You've got yourself a delicious \ 
cup of tea to enjoy.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 1:")
print(response)

Completion for Text 1:

Step 1 - Get some water boiling
Step 2 - Grab a cup and put a tea bag in it
Step 3 - Once the water is hot enough, just pour it over the tea bag
Step 4 - Let it sit for a bit so the tea can steep
Step 5 - Take out the tea bag
Step 6 - If you like, you can add some sugar or milk to taste
Step 7 - And that's it! You've got yourself a delicious cup of tea to enjoy.


In [35]:
text_2 = f"""
The sun is shining brightly today, and the birds are \
singing. It's a beautiful day to go for a \ 
walk in the park. The flowers are blooming, and the \ 
trees are swaying gently in the breeze. People \ 
are out and about, enjoying the lovely weather. \ 
Some are having picnics, while others are playing \ 
games or simply relaxing on the grass. It's a \ 
perfect day to spend time outdoors and appreciate the \ 
beauty of nature.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 2:")
print(response)

Completion for Text 2:

Step 1 - The sun is shining brightly today, and the birds are singing. It's a beautiful day to go for a walk in the park.
Step 2 - The flowers are blooming, and the trees are swaying gently in the breeze. People are out and about, enjoying the lovely weather.
Step 3 - Some are having picnics, while others are playing games or simply relaxing on the grass. It's a perfect day to spend time outdoors and appreciate the beauty of nature.


#### Tactic 4: "Few-shot" prompting

In [19]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest \ 
valley flows from a modest spring; the \ 
grandest symphony originates from a single note; \ 
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""
response = get_completion(prompt)
print(response)


<grandparent>: Resilience is the ability to bounce back from a difficult situation. It is the ability to recover from a challenge or a set-back. It is the ability to keep going, even when things are tough.

Resilience is a key skill for success in life. It is the ability to keep going, even when things are tough. It is the ability to recover from a challenge or a set-back. It is the ability to bounce back from a difficult situation.


### Principle 2: Give the model time to “think” 

#### Tactic 1: Specify the steps required to complete a task

In [29]:
text = f"""
In a charming village, siblings Jack and Jill set out on \ 
a quest to fetch water from a hilltop \ 
well. As they climbed, singing joyfully, misfortune \ 
struck—Jack tripped on a stone and tumbled \ 
down the hill, with Jill following suit. \ 
Though slightly battered, the pair returned home to \ 
comforting embraces. Despite the mishap, \ 
their adventurous spirits remained undimmed, and they \ 
continued exploring with delight.
"""
# example 1
prompt_1 = f"""
Perform the following actions: 
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following \
keys: french_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
1 - Summarize the following text delimited by triple backticks with 1 sentence.
The story of two siblings, Jack and Jill, who went to fetch water from a hilltop well, and the misfortunes that befell them along the way.
2 - Translate the summary into French.
La légende de deux frères et sœurs, Jack et Jill, qui allaient chercher l'eau à la fontaine du sommet d'une colline, et les malheurs qui les accablaient tout au long de la route.
3 - List each name in the French summary.
Jack, Jill
4 - Output a json object that contains the following keys: french_summary, num_names.
{'french_summary': 'La légende de deux frères et sœurs, Jack et Jill, qui allaient chercher l'eau à la fontaine du sommet d'une colline, et les malheurs qui les accablaient tout au long de la route.', 'num_names': 2}


#### Ask for output in a specified format

In [34]:
prompt_2 = f"""
Your task is to perform the following actions: 
1 - Summarize the following text delimited by 
  <> with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the 
  following keys: french_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Output JSON: <json with summary and num_names>
Names: <list of names in Italian summary>

Text: <{text}>
"""
response = get_completion(prompt_2)
print("\nCompletion for prompt 2:")
print(response)


Completion for prompt 2:
Text: <
In a charming village, siblings Jack and Jill set out on a quest to fetch water from a hilltop well. As they climbed, singing joyfully, misfortune struck—Jack tripped on a stone and tumbled down the hill, with Jill following suit. Though slightly battered, the pair returned home to comforting embraces. Despite the mishap, their adventurous spirits remained undimmed, and they continued exploring with delight.
>
Summary: <
In a charming village, siblings Jack and Jill set out on a quest to fetch water from a hilltop well. As they climbed, singing joyfully, misfortune struck—Jack tripped on a stone and tumbled down the hill, with Jill following suit. Though slightly battered, the pair returned home to comforting embraces. Despite the mishap, their adventurous spirits remained undimmed, and they continued exploring with delight.
>
Translation: <
In un village charmant, les frères et sœurs Jack et Jill démarrent une quête pour aller chercher l'eau à une fon

#### Tactic 2: Instruct the model to work out its own solution before rushing to a conclusion

In [36]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \ 
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt)
print(response)

The student's solution is correct.


#### Note that the student's solution is actually not correct.
#### We can fix this by instructing the model to work out its own solution first.

In [57]:
prompt = f"""
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem. 
- Then compare your solution to the student's solution \ 
and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until 
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help \
working out the financials. 
- Land costs $100 per square foot
- I can buy solar panels for $250 per square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 per square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
``` 
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 10x = 360x + 100,000
```
Actual solution:
"""
response = get_completion(prompt)
print(response)

Using the same variable names as the student, we can write the equation as:

Total cost = 100x + 250x + 100,000 + 10x = 360x + 100,000

Solving for x:

x = 360,000 - 100,000 - 250,000 - 100,000 - 10,000 = 150,000

So the total cost for the first year of operations is $5,100,000.

Is the student's solution the same as the actual solution? Yes.

Student grade: Correct.


## Model Limitations: Hallucinations
- Boie is a real company, the product name is not real.

In [58]:
prompt = f"""
Tell me about AeroGlide UltraSlim Smart Toothbrush by Boie
"""
response = get_completion(prompt)
print(response)

The AeroGlide UltraSlim Smart Toothbrush by Boie is a high-quality electric toothbrush that provides a comfortable and effective brushing experience. The toothbrush features a slim and lightweight design, making it easy to use and comfortable to hold. The toothbrush also has a built-in timer that helps ensure you are brushing for the recommended 2 minutes. The toothbrush is powered by a rechargeable battery, so you don't have to worry about replacing batteries.


## Try experimenting on your own!

#### Notes on using the cohere API outside of this classroom

To install the cohere Python library:
```
!pip install cohere
```

The library needs to be configured with your account's secret key, which is available on the [website](https://dashboard.cohere.ai/api-keys). 

You can either set it as the `OPENAI_API_KEY` environment variable before using the library:
 ```
 !export COHERE_API_KEY='sk-...'
 ```

Or, set `cohere.api_key` to its value:

```
import cohere
cohere.api_key = "..."
```

#### A note about the backslash
- In the course, we are using a backslash `\` to make the text fit on the screen without inserting newline '\n' characters.
- GPT-3 isn't really affected whether you insert newline characters or not.  But when working with LLMs in general, you may consider whether newline characters in your prompt may affect the model's performance.

# 2. Iterative Prompt Develelopment
In this lesson, you'll iteratively analyze and refine your prompts to generate marketing copy from a product fact sheet.

## Setup

## Generate a marketing product description from a product fact sheet

In [59]:
fact_sheet_chair = """
OVERVIEW
- Part of a beautiful family of mid-century inspired office furniture, 
including filing cabinets, desks, bookcases, meeting tables, and more.
- Several options of shell color and base finishes.
- Available with plastic back and front upholstery (SWC-100) 
or full upholstery (SWC-110) in 10 fabric and 6 leather options.
- Base finish options are: stainless steel, matte black, 
gloss white, or chrome.
- Chair is available with or without armrests.
- Suitable for home or business settings.
- Qualified for contract use.

CONSTRUCTION
- 5-wheel plastic coated aluminum base.
- Pneumatic chair adjust for easy raise/lower action.

DIMENSIONS
- WIDTH 53 CM | 20.87”
- DEPTH 51 CM | 20.08”
- HEIGHT 80 CM | 31.50”
- SEAT HEIGHT 44 CM | 17.32”
- SEAT DEPTH 41 CM | 16.14”

OPTIONS
- Soft or hard-floor caster options.
- Two choices of seat foam densities: 
 medium (1.8 lb/ft3) or high (2.8 lb/ft3)
- Armless or 8 position PU armrests 

MATERIALS
SHELL BASE GLIDER
- Cast Aluminum with modified nylon PA6/PA66 coating.
- Shell thickness: 10 mm.
SEAT
- HD36 foam

COUNTRY OF ORIGIN
- Italy
"""

In [69]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)


Mid-century inspired office chair with a beautiful family of mid-century inspired office furniture. Several options of shell color and base finishes. Available with plastic back and front upholstery (SWC-100) or full upholstery (SWC-110) in 10 fabric and 6 leather options. Base finish options are: stainless steel, matte black, gloss white, or chrome. Chair is available with or without armrests. Suitable for home or business settings. Qualified for contract use.


## Issue 1: The text is too long 
- Limit the number of words/sentences/characters.

In [75]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)


Mid-century inspired office chair.

Available with plastic back and front upholstery (SWC-100) or full upholstery (SWC-110) in 10 fabric and 6 leather options.

Base finish options are: stainless steel, matte black, gloss white, or chrome.

Chair is available with or without armrests.

 Suitable for home or business settings.

Qualified for contract use.

Width 53 CM | 20.87”
Depth 51 CM | 20.08”
Height 80 CM | 31.50”
Seat Height 44 CM | 17.32”
Seat Depth 41 CM | 16.14”

Options:
Soft or hard-floor caster options.
Two choices of seat foam densities: medium (1.8 lb/ft3) or high (2.8 lb/ft3).
Armless or 8 position PU armrests 

Materials:
Shell Base Glider: Cast Aluminum with modified nylon PA6/PA66 coating.
Shell thickness: 10 mm.
Seat: HD36 foam

Country of Origin:
Italy


In [76]:
len(response.split(" "))

108

## Issue 2. Text focuses on the wrong details
- Ask it to focus on the aspects that are relevant to the intended audience.

In [77]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Mid-century inspired office chair.
- 5-wheel plastic coated aluminum base.
- Pneumatic chair adjust for easy raise/lower action.
- Seat with HD36 foam.
- Cast Aluminum with modified nylon PA6/PA66 coating.
- Shell thickness: 10 mm.
- Suitable for home or business settings.
- Qualified for contract use.
- Armless or 8 position PU armrests.
- Two choices of seat foam densities: medium (1.8 lb/ft3) or high (2.8 lb/ft3).
- Soft or hard-floor caster options.
- Available with plastic back and front upholstery (SWC-100) 
or full upholstery (SWC-110) in 10 fabric and 6 leather options.
- Base finish options are: stainless steel, matte black, 
gloss white, or chrome.


In [78]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Technical specifications:

OVERVIEW
- Part of a beautiful family of mid-century inspired office furniture, including filing cabinets, desks, bookcases, meeting tables, and more.
- Several options of shell color and base finishes.
- Available with plastic back and front upholstery (SWC-100) or full upholstery (SWC-110) in 10 fabric and 6 leather options.
- Base finish options are: stainless steel, matte black, gloss white, or chrome.
- Chair is available with or without armrests.
- Suitable for home or business settings.
- Qualified for contract use.

CONSTRUCTION
- 5-wheel plastic coated aluminum base.
- Pneumatic chair adjust for easy raise/lower action.

DIMENSIONS
- WIDTH 53 CM | 20.87”
- DEPTH 51 CM | 20.08”
- HEIGHT 80 CM | 31.50”
- SEAT HEIGHT 44 CM | 17.32”
- SEAT DEPTH 41 CM | 16.14”

OPTIONS
- Soft or hard-floor caster options.
- Two choices of seat foam densities: medium (1.8 lb/ft3) or high (2.8 lb/ft3)
- Armless or 8 position PU armrests 

MATERIALS
SHELL BASE GLIDER
- Cast

## Issue 3. Description needs a table of dimensions
- Ask it to extract information and organize it in a table.

In [79]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

After the description, include a table that gives the 
product's dimensions. The table should have two columns.
In the first column include the name of the dimension. 
In the second column include the measurements in inches only.

Give the table the title 'Product Dimensions'.

Format everything as HTML that can be used in a website. 
Place the description in a <div> element.

Technical specifications: ```{fact_sheet_chair}```
"""

response = get_completion(prompt)
print(response)

<div>
The SWC-100/110 is a beautiful mid-century inspired office chair. It is part of a family of office furniture that includes filing cabinets, desks, bookcases, meeting tables, and more. The SWC-100/110 is available with a plastic back and front upholstery (SWC-100) or full upholstery (SWC-110) in 10 fabric and 6 leather options. The chair is available with or without armrests. The base finish options are: stainless steel, matte black, gloss white, or chrome. The chair is suitable for home or business settings and is qualified for contract use.

The SWC-100/110 has a 5-wheel plastic coated aluminum base. It has a pneumatic chair adjust for easy raise/lower action.

The chair's dimensions are: WIDTH 53 CM | 20.87”, DEPTH 51 CM | 20.08”, HEIGHT 80 CM | 31.50”, SEAT HEIGHT 44 CM | 17.32”, SEAT DEPTH 41 CM | 16.14”.

The SWC-100/110 is made in Italy.
</div>

<table class="product_dimensions">
	<tr>
		<th>Dimension</th>
		<th>Value</th>
	</tr>
	<tr>
		<td>Width</td>
		<td>53 cm | 20.87</

## Load Python libraries to view HTML

In [80]:
from IPython.display import display, HTML

In [81]:
display(HTML(response))

Dimension,Value
Width,53 cm | 20.87
Depth,51 cm | 20.08
Height,80 cm | 31.50
Seat Height,44 cm | 17.32
Seat Depth,41 cm | 16.14


## Try experimenting on your own!